In [1]:
import os
import pathlib
from urllib.request import urlretrieve
from time import sleep
import math as math
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.ticker as ticker

import sys
sys.path.append('/code')
from fdmonitor.cont_futures import create_continuous_future, get_cme_data
from fdmonitor.plotter import draw_charts

%matplotlib inline

In [2]:
cme_metadata = pd.read_csv('CME_metadata.csv')
cme_futures = pd.read_csv('CME_futures.csv')
cme_futures.set_index('Futures Symbol', inplace=True)
DATA_DIRNAME = pathlib.Path('/code').resolve() / 'data'

In [3]:
futures_symbol = 'ES'
futures_name, CME_months, CME_years = get_cme_data(futures_symbol, cme_futures, cme_metadata)

In [4]:
data = create_continuous_future(futures_symbol, DATA_DIRNAME, CME_months, CME_years,
                                        'open_interest', False)

In [5]:
def create_pdf(futures_symbol, cme_futures, cme_metadata):
    futures_name, CME_months, CME_years = get_cme_data(futures_symbol, 
                                                       cme_futures, cme_metadata)
    roll_rules = ['expiration', 'open_interest', 'exp_month']
    true_false = [False, True]

    pdf_pages = PdfPages(pathlib.Path('/code/pdfs/tmp').resolve() / f'{futures_symbol}.pdf')

    for roll_rule in roll_rules:
        for tf in true_false:
            data = create_continuous_future(futures_symbol, DATA_DIRNAME, CME_months, CME_years,
                                        roll_rule, tf)
            plot_title = f'{futures_name} ({futures_symbol}) \n Roll: {roll_rule}'
            if tf:
                plot_title = plot_title + ', backadjusted'
            draw_charts(data, plot_title, pdf_pages)
    pdf_pages.close()

In [6]:
create_pdf('NQ', cme_futures, cme_metadata)

In [7]:
for futures_symbol in cme_futures.index.tolist():
    create_pdf(futures_symbol, cme_futures, cme_metadata)

/usr/local/lib/python3.6/site-packages/matplotlib/tight_layout.py:199: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  warnings.warn('Tight layout not applied. '
